In [1]:
import tensorflow as tf
from keras import backend as K

sess = tf.Session()

from keras import backend as K
K.set_session(sess)

Using TensorFlow backend.


In [2]:
from datagenerator_tfrecord import ImageDataGenerator
Iterator = tf.data.Iterator

filewriter_path = 'summary/'
checkpoint_path = 'checkpoints/'
example_path = '/home/aldin/Desktop/train.tfrecords'

# learning_rate = 0.01
num_epochs = 20
batch_size = 64
percent_for_train = 0.98
_WEIGHT_DECAY = 1e-4
_MOMENTUM = 0.9
num_classes = 5270

with tf.device('/cpu:0'):
    tr_data = ImageDataGenerator(example_path,
                                 mode='training',
                                 batch_size=batch_size,
                                 num_classes=num_classes,
                                 shuffle=True)
#     val_data = ImageDataGenerator(example_path,
#                                   mode='validation',
#                                   batch_size=batch_size,
#                                   num_classes=num_classes,
#                                   shuffle=False)
    # create an reinitializable iterator given the dataset structure
    iterator = Iterator.from_structure(tr_data.data.output_types,
                                       tr_data.data.output_shapes)
    x, y = iterator.get_next()
    
# Ops for initializing the two different iterators
training_init_op = iterator.make_initializer(tr_data.data)
# validation_init_op = iterator.make_initializer(val_data.data)

In [4]:
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.layers import Flatten, Dense

# model = Xception(include_top=False, weights='imagenet', input_tensor=x, input_shape=(160, 160, 3), pooling='avg', classes=None)
model = ResNet50(include_top=False, weights='imagenet', input_shape=(200, 200, 3))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 100, 100, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 100, 100, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_50 (Activation)      (None, 100, 100, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [3]:
from keras.models import Model
from keras.layers import Flatten, Dense

for layer in model.layers:
    layer.trainable=False

# X = Flatten()(model.output)
# X = Dense(5270, activation='relu')(model.output)
logits = Dense(5270, activation='softmax')(model.output)
new_model = Model(input = model.input, output = logits)
new_model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'], target_tensors=[y])
new_model.summary()

/home/aldin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


NameError: name 'y' is not defined

In [5]:
sess.run(training_init_op)
new_model.fit(steps_per_epoch=1000, epochs=5, verbose=2)

Epoch 1/5


KeyboardInterrupt: 